In [1]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
import time

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)
matplotlib 3.2.2
numpy 1.19.0
pandas 1.0.5
sklearn 0.23.1
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
# 根据cifar10的数据内容
class_names = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck',
]
# 本地数据集路径
train_labels_file = './cifar10/trainLabels.csv'
test_csv_file = './cifar10/sampleSubmission.csv'
train_folder = './cifar10/train/'
test_folder = './cifar10/test/'

print(os.path.exists(train_folder))
print(os.path.exists(test_folder))
print(os.path.exists(train_labels_file))


# 读取数据，组装数据
# Parses csv files into (filename(path),label) format
def parse_csv_file(filepath, folder):
    """Parses csv files into (filename(path),label) format"""
    results = []
    with open(filepath, 'r') as f:
        lines = f.readlines()[1:] #不需要header 
    for line in lines:
        image_id, label_str = line.strip('\n').split(',')
        image_full_path = os.path.join(folder,image_id +'.png')
        results.append((image_full_path,label_str))
    return results

train_labels_info = parse_csv_file(train_labels_file,train_folder)
test_csv_info = parse_csv_file(test_csv_file,test_folder)

import pprint
pprint.pprint(train_labels_info[0:5])
pprint.pprint(test_csv_info[0:5])
print(len(train_labels_info), len(test_csv_info))


True
True
True
[('./cifar10/train/1.png', 'frog'),
 ('./cifar10/train/2.png', 'truck'),
 ('./cifar10/train/3.png', 'truck'),
 ('./cifar10/train/4.png', 'deer'),
 ('./cifar10/train/5.png', 'automobile')]
[('./cifar10/test/1.png', 'cat'),
 ('./cifar10/test/2.png', 'cat'),
 ('./cifar10/test/3.png', 'cat'),
 ('./cifar10/test/4.png', 'cat'),
 ('./cifar10/test/5.png', 'cat')]
50000 300000


In [3]:
# train_df = pd.DataFrame(train_labels_info)
train_df = pd.DataFrame(train_labels_info[0:45000])
valid_df = pd.DataFrame(train_labels_info[45000:])
test_df = pd.DataFrame(test_csv_info)
# 添加列名
train_df.columns = ['filepath','class']
valid_df.columns = ['filepath','class']
test_df.columns = ['filepath','class']
print(train_df.head())
print(valid_df.head())
print(test_df.head())

                filepath       class
0  ./cifar10/train/1.png        frog
1  ./cifar10/train/2.png       truck
2  ./cifar10/train/3.png       truck
3  ./cifar10/train/4.png        deer
4  ./cifar10/train/5.png  automobile
                    filepath       class
0  ./cifar10/train/45001.png       horse
1  ./cifar10/train/45002.png  automobile
2  ./cifar10/train/45003.png        deer
3  ./cifar10/train/45004.png  automobile
4  ./cifar10/train/45005.png    airplane
               filepath class
0  ./cifar10/test/1.png   cat
1  ./cifar10/test/2.png   cat
2  ./cifar10/test/3.png   cat
3  ./cifar10/test/4.png   cat
4  ./cifar10/test/5.png   cat


In [4]:
# 卷积神经网络无法预处理大小不一致的图片，所以要对读取的图片进行大小变化
# cifar10 图片是  32*32
height = 32
width = 32
channels = 3 
batch_size = 32
num_classes = 10
epochs = 50

# 读取训练数据并作数据增强
# 确定一些读取格式要求
# keras.preprocessing.image.ImageDataGenerator
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    # 图片旋转的角度范围，用来数据增强
    rotation_range = 40,
    # 水平平移
    width_shift_range = 0.2,
    # 高度平移
    height_shift_range = 0.2,
    # 剪切强度
    shear_range = 0.2,
    # 缩放强度
    zoom_range = 0.2,
    # 水平翻转
    horizontal_flip = True,
    # 对图片做处理时需要填充图片，用最近的像素点填充
    fill_mode = "nearest"
)
# 读取训练数据
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory = './',
    x_col = 'filepath',
    y_col = 'class',
    classes = class_names, # 字符类型映射
    # 读取后将图片存什么大小 
    target_size = (height, width),
    batch_size = batch_size,
    seed = 7,
    shuffle = True,
    # label的编码格式：字符类型
    class_mode = 'sparse'
)

# 读取验证数据
valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
valid_generator = valid_datagen.flow_from_dataframe(
    valid_df,
    directory = './',
    x_col = 'filepath',
    y_col = 'class',
    classes = class_names,
    # 读取后将图片存什么大小 
    target_size = (height, width),
    batch_size = batch_size,
    seed = 7,
    shuffle = False,
    # label的编码格式：
    class_mode = 'sparse')

train_num = train_generator.samples
valid_num = valid_generator.samples
print(train_num, valid_num)

Found 45000 validated image filenames belonging to 10 classes.
Found 5000 validated image filenames belonging to 10 classes.
45000 5000


In [5]:
for i in range(2):
    x,y = train_generator.next()
    print(x.shape,y.shape)
    print(y)
    


(32, 32, 32, 3) (32,)
[2. 1. 4. 4. 4. 4. 6. 5. 2. 8. 4. 6. 6. 3. 7. 1. 7. 2. 8. 8. 3. 0. 5. 3.
 9. 1. 4. 5. 6. 7. 9. 2.]
(32, 32, 32, 3) (32,)
[0. 7. 2. 7. 5. 5. 7. 0. 5. 4. 9. 7. 6. 3. 0. 4. 4. 4. 6. 3. 5. 4. 6. 6.
 4. 1. 8. 2. 4. 4. 3. 0.]


In [6]:
# keras.layers.Conv2D
# keras.layers.BatchNormalization
# keras.layers.MaxPool2D
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation='relu',input_shape=[height,width,channels]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(filters=512,kernel_size=3,padding='same',activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=512,kernel_size=3,padding='same',activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(512,activation='relu'),
    keras.layers.Dense(num_classes,activation='softmax')
    
])
# 因为y是one-hot 所以 categorical_crossentropy  非 sparse_categorical_crossentropy
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy']) 

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       147584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 256)       1

In [7]:
logdir = 'cifar10-cnn-callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)    
output_model_file = os.path.join(logdir,"cifar10_cnn_model.h5")

callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file,save_best_only=True),
    keras.callbacks.EarlyStopping(patience=5,min_delta=1e-8)
]
# model.fit_generator
history = model.fit_generator(train_generator,
                               steps_per_epoch = train_num // batch_size,
                               epochs = epochs,
                               validation_data = valid_generator,
                               validation_steps = valid_num // batch_size,
                               callbacks=callbacks)


Epoch 1/50
 631/1406 [============>.................] - ETA: 34:05 - loss: 3.3370 - accuracy: 0.2247

KeyboardInterrupt: 

In [ ]:
history.history.keys()

In [ ]:
def plot_leraning_curves(history, label, epochs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val_'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()
    
plot_leraning_curves(history, 'accuracy', epochs, 0, 1)
plot_leraning_curves(history, 'loss', epochs, 1.0, 2.5)

In [ ]:
# 读取测试数据
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
test_generator = valid_datagen.flow_from_dataframe(
    test_df,
    directory = './',
    x_col = 'filepath',
    y_col = 'class',
    classes = class_names,
    # 读取后将图片存什么大小 
    target_size = (height, width),
    batch_size = batch_size,
    seed = 7,
    shuffle = False,
    # label的编码格式：
    class_mode = 'sparse')

test_num = test_generator.samples
print(test_num)

In [ ]:
# model.predict_generator
test_predict = model.predict_generator(test_generator,workers =10,use_multiprocessing = False)
print(test_predict.shape)

In [ ]:
print(test_predict[0:5])

In [ ]:
test_predict_class_indices = np.argmax(test_predict,axis=1) # axis=1行  0列
print(test_predict_class_indices[0:5])

In [ ]:
test_predict_class = [class_names[index] for index in test_predict_class_indices] #根据index获取class name
print(test_predict_class[0:5])

In [ ]:
# 预测结果写入文件
def generate_submissions(filename,predict_class):
    with open(filename,'w') as f:
        f.write('id,label\n')
        for i in range(len(predict_class)):
            f.write('%d,%s\n'% (i+1,predict_class[i])) # i 从0开始
            
output_file = "./cifar10/submission.csv"
generate_submissions(output_file,test_predict_class)